In [1]:
'''
ryu-lab-06-2-softmax_zoo_classifier-DNN 를 수정해서 IVR서비스 추천을 위한 Collaborative Filtering 으로 사용
2017.10.9
류기동
'''
import tensorflow as tf
import numpy as np
import datetime

In [2]:

tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
#xy = np.loadtxt('./IVRService/CfDataSmall-20171009.csv', delimiter=',', dtype=np.float32)
xy = np.loadtxt('./IVRService/CfData-Y01-20171015.csv', delimiter=',', dtype=np.float32)
'''
CfDataSmall 은 전체 중에 1006개의 레코드만 가진 샘플 데이터
UserID, ServiceCODE, 사용여부 3개의 컬럼 가지고 있음
Y=0 인것의 갯수 7932, Y=1 인것의 갯수 7932
record수는 15863 개
2,100900,0
2,100901,0
2,100902,0
2,100903,1
2,100904,0

'''

'\nCfDataSmall 은 전체 중에 1006개의 레코드만 가진 샘플 데이터\nUserID, ServiceCODE, 사용여부 3개의 컬럼 가지고 있음\nY=0 인것의 갯수 7932, Y=1 인것의 갯수 7932\nrecord수는 15863 개\n2,100900,0\n2,100901,0\n2,100902,0\n2,100903,1\n2,100904,0\n\n'

In [3]:
x_data = xy[:, 0:-1]  #UserID, Service Code
y_data = xy[:, [-1]]  #서비스 사용여부

print(x_data.shape, y_data.shape)

print (x_data.shape[1])

x1_data = x_data[:, [0]]  #UserID부분  => Embedding Layer의 입력
x2_data = x_data[:, [1]]  #Service Code 부분 => Embedding Layer의 입력

(15864, 2) (15864, 1)
2


In [4]:
# one-hot 으로 데이터 변환
#X1은 User ID : 234가지 종류, X2는 Service Code,  43가지
x_col = x_data.shape[1]
y_col = y_data.shape[1]
print (x_col, y_col)


2 1


In [5]:

#X = tf.placeholder(tf.float32, [None, x_col])  #이거는 필요 없을듯
X1 = tf.placeholder(tf.int32, [None, 1])  #one hot 을 할려면 int32로 해야 한다. float32는 안된다.
X2 = tf.placeholder(tf.int32, [None, 1])
Y = tf.placeholder(tf.int32, [None, y_col])  # 


In [6]:
#Y_one_hot = tf.one_hot(Y, 7)

In [7]:
#one-hot 으로 변경 하기
#X1_classes = 234
X1_classes = 2495
X2_classes = 43
Y_OUT = 2
X1_one_hot = tf.one_hot(X1, X1_classes)  # one hot
print("X1 one_hot", X1_one_hot)
X1_one_hot = tf.reshape(X1_one_hot, [-1, X1_classes])
print("X1 reshape", X1_one_hot)
X2_one_hot = tf.one_hot(X2, X2_classes)  # one hot
print("X2 one_hot", X2_one_hot)
X2_one_hot = tf.reshape(X2_one_hot, [-1, X2_classes])
print("X2 reshape", X2_one_hot)

Y_one_hot = tf.one_hot(Y, Y_OUT)  # one hot
print("Y one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, Y_OUT])
print("Y reshape", Y_one_hot)



X1 one_hot Tensor("one_hot:0", shape=(?, 1, 2495), dtype=float32)
X1 reshape Tensor("Reshape:0", shape=(?, 2495), dtype=float32)
X2 one_hot Tensor("one_hot_1:0", shape=(?, 1, 43), dtype=float32)
X2 reshape Tensor("Reshape_1:0", shape=(?, 43), dtype=float32)
Y one_hot Tensor("one_hot_2:0", shape=(?, 1, 2), dtype=float32)
Y reshape Tensor("Reshape_2:0", shape=(?, 2), dtype=float32)


In [8]:
EL_OUT_1 = 256
EL_OUT_2 = 32
HIDDEN_OUT_1 = 64
HIDDEN_OUT_2 = 64
HIDDEN_OUT_3 = 64
HIDDEN_OUT_4 = 64
HIDDEN_OUT_5 = 64


# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

#Embedding Layer EL1 은 X1의 Embedding, EL2는 X2의 Embedding
#Embedding을 하고 나면 차원이 줄어든다. 
# EL1 = X1*WE1 + bE1
# EL2 = X2*WE2 + bE2
#EL1, EL2가 실질적인 INPUT 값 X의 역할을 한다고 보면 된다., 겁나큰 스파스한 벡터를 덴스하게 바꿔서 MLP에 
#입력으로 사용한다는 의미

#Layer1(첫번째) 에서는 EL1, EL2를 합쳐서 받아야 하므로 
# L1 = (EL1 + EL2) * W1 + b1 이 된다. 
# L2 = L1 * W2 + b2
#....
'''
WEL1  이 어쨌든 첫번째 입력 단이다. 여기서 X1은 x1_data 가 feed 된다. x1_data 는 X1_one_hot으로 변경된다.
X1_one_hot은 ?, 234 이므로 WEL1(EL1의 Weight) 의 shape는 234, EL_OUT_1(32) 가 된다. 즉 234차원을 32차원으로 
변형(임베딩) 하는 것이다.

'''
with tf.name_scope("Embedding_Layer_User") as scope :
    WEL1 = tf.get_variable("WEL1", shape=[X1_classes, EL_OUT_1], initializer=tf.contrib.layers.xavier_initializer() )
    bE1 = tf.Variable(tf.random_normal([EL_OUT_1]))
    #LE1 = tf.nn.relu(tf.matmul(X1_one_hot, WEL1) + bE1)
    LE1 = tf.matmul(X1_one_hot, WEL1) + bE1
    

with tf.name_scope("Embedding_Layer_Service") as scope :
    WEL2 = tf.get_variable("WEL2", shape=[X2_classes, EL_OUT_2], initializer=tf.contrib.layers.xavier_initializer() )
    bE2 = tf.Variable(tf.random_normal([EL_OUT_2]))
    #LE2 = tf.nn.relu(tf.matmul(X2_one_hot, WEL2) + bE2)
    LE2 = tf.matmul(X2_one_hot, WEL2) + bE2



In [9]:
'''
L1은 EL1, EL2가 합쳐지는 계층, MLP에서는 첫번째 Hidden 이 된다. 
입력은 EL1 + EL2 크기이고 출력은 HIDDEN_OUT_1 이 된다.
입력 값은 LE1 과 LE2를 tf.contact 해서 행렬을 맞춘다.
'''
with tf.name_scope("Input_Layer") as scope :
    L1_in = tf.concat([LE1, LE2],1)
    print(L1_in.shape) # 32+ 16 = 48
    print(LE1.shape)

(?, 288)
(?, 256)


In [10]:
#L1_in.shape
with tf.name_scope("Layer1") as scope :
    W1 = tf.get_variable("W1", shape=[EL_OUT_1+EL_OUT_2, HIDDEN_OUT_1], initializer=tf.contrib.layers.xavier_initializer() )
    b1 = tf.Variable(tf.random_normal([HIDDEN_OUT_1]))
    L1 = tf.nn.relu(tf.matmul(L1_in, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


In [11]:
with tf.name_scope("Layer2") as scope :
    W2 = tf.get_variable("W2", shape=[HIDDEN_OUT_1,HIDDEN_OUT_2], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([HIDDEN_OUT_2]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
with tf.name_scope("Layer3") as scope :
    W3= tf.get_variable("W3", shape=[HIDDEN_OUT_2,HIDDEN_OUT_3], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([HIDDEN_OUT_3]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
with tf.name_scope("Layer4") as scope :
    W4= tf.get_variable("W4", shape=[HIDDEN_OUT_3,HIDDEN_OUT_4], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([HIDDEN_OUT_4]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
    
with tf.name_scope("Layer5") as scope :
    W5= tf.get_variable("W5", shape=[HIDDEN_OUT_4,HIDDEN_OUT_5], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([HIDDEN_OUT_5]))
    L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

In [12]:
#Last Layer (FC) and Hypothesis
#variable 이름이 들어가는 코드는 컴파일을 다시 할 수 없다. 
with tf.name_scope("FC_Layer") as scope :
    WFC = tf.get_variable("WFC", shape=[HIDDEN_OUT_5, Y_OUT],
                         initializer=tf.contrib.layers.xavier_initializer())
    bFC = tf.Variable(tf.random_normal([Y_OUT]))
    logit = tf.matmul(L5, WFC) + bFC
    hypothesis = tf.nn.softmax(logit)
#hypothesis = tf.matmul(L3, WFC) + bFC
#hypothesis = tf.sigmoid(tf.matmul(L3, WFC) + bFC)

In [13]:
# parameters
learning_rate = 0.005
training_epochs = 70000
batch_size = 500 # batch size는 크면 학습량이 작다

#logit = tf.matmul(L3, WFC) + bFC
#hypothesis = tf.nn.softmax(logit)
#define cost/loss & optimizer
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#softmax 는 logit을 입력으로 받는다. logit은 0~1까지 확률값을 가진값이다.
#예을 들어 y_one_hot 이 2개 클래스에 대한 값이면 logit은 각각에 대해 0~1까지 확률 값이 나오고
#softmax는 그값을 받아서 2개 클래스 중에 한개 값이 많이 크게 만들어 준다.
with tf.name_scope("Cost") as scope :
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y_one_hot))
    cost_str = tf.summary.scalar("Cost", cost)
#cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
#                       tf.log(1 - hypothesis))
with tf.name_scope("Train") as scope :
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [14]:
x1_data.shape[0]

15864

In [15]:
print('[%s]'% datetime.datetime.now(), 'Epoch:', '%04d' % (1))

[2018-01-19 21:50:29.135134] Epoch: 0001


In [16]:
#학슴 하면서 테스트 할 것 들 있으면 계산식을 미리 선언한다.
prediction = tf.argmax(hypothesis, 1)
correction_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))
acc_str = tf.summary.scalar("Accuracy", accuracy)


# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary = tf.summary.merge_all()
#merged = tf.merge_all_summaries()
writer = tf.summary.FileWriter("./logs/test", sess.graph )
#writer = tf.summary.FileWriter("./logs/test", sess.graph_def )
#writer.add_graph(sess.graph)

n_deg = 100
# train my model
print('[%s]'% datetime.datetime.now(), ' Start Learning !!')

for epoch in range(training_epochs):
   
    avg_cost = 0
    avg_acc = 0
    total_batch = int(x1_data.shape[0] / batch_size)  #10062/batch_size
    #total_batch = int(x1_data.shape[0]) # 101개 밖에 안되니까 그냥 다 돌린다.
    
    ############################
    #Mini batch 구간
    #for i in range(total_batch):  #미니 배치를 돌릴려면 feed_dict에 데이터도 짤라야 된다.
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    feed_dict = {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 0.7}
    c, acc,  _ = sess.run([cost, accuracy,optimizer], feed_dict=feed_dict)
    #avg_cost += c / total_batch
    '''
        if i % 50  == 0 : 
            loss, acc = sess.run([cost, accuracy], feed_dict= {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
            print('BatchRun:', '%03d'%(i+1), 'cost=', '{:.9f}'.format(loss),
                 'acc=', '{:.9f}'.format(acc))
    '''
        #if i % 25  == 0 : print('BatchRun:', '%03d'%(i+1), 'cost=', '{:.9f}'.format(avg_cost))
      # datetime.datetime.now() 
    
    
    if epoch % n_deg == 0 : 
        print('[%s]'% datetime.datetime.now(), 'Epoch:', '%04d' % (epoch + 1), ' Learning Done!!')
        print('[%s]'% datetime.datetime.now(), 'Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(c),'acc =', '{:.9f}'.format(acc) )
        acc, summary_str = sess.run([accuracy, summary], feed_dict = {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
        writer.add_summary(summary_str, epoch)

print('Learning Finished!')



[2018-01-19 21:50:33.078562]  Start Learning !!
[2018-01-19 21:50:35.638393] Epoch: 0001  Learning Done!!
[2018-01-19 21:50:35.638393] Epoch: 0001 cost = 1.248354793 acc = 0.495398372
[2018-01-19 21:54:39.523506] Epoch: 0101  Learning Done!!
[2018-01-19 21:54:39.523506] Epoch: 0101 cost = 0.693724513 acc = 0.505736232
[2018-01-19 21:58:43.941641] Epoch: 0201  Learning Done!!
[2018-01-19 21:58:43.941641] Epoch: 0201 cost = 0.693202138 acc = 0.505042851
[2018-01-19 22:02:48.195313] Epoch: 0301  Learning Done!!
[2018-01-19 22:02:48.195313] Epoch: 0301 cost = 0.693324268 acc = 0.497289449
[2018-01-19 22:06:52.503093] Epoch: 0401  Learning Done!!
[2018-01-19 22:06:52.503093] Epoch: 0401 cost = 0.693183064 acc = 0.502017140
[2018-01-19 22:10:56.755619] Epoch: 0501  Learning Done!!
[2018-01-19 22:10:56.755619] Epoch: 0501 cost = 0.693145335 acc = 0.500945508
[2018-01-19 22:15:00.741690] Epoch: 0601  Learning Done!!
[2018-01-19 22:15:00.741690] Epoch: 0601 cost = 0.693244755 acc = 0.492813915


[2018-01-20 01:54:47.176839] Epoch: 6001  Learning Done!!
[2018-01-20 01:54:47.176839] Epoch: 6001 cost = 0.688883364 acc = 0.509014130
[2018-01-20 01:58:51.242544] Epoch: 6101  Learning Done!!
[2018-01-20 01:58:51.242544] Epoch: 6101 cost = 0.689011753 acc = 0.508509815
[2018-01-20 02:02:55.283274] Epoch: 6201  Learning Done!!
[2018-01-20 02:02:55.283274] Epoch: 6201 cost = 0.688991606 acc = 0.508888066
[2018-01-20 02:06:59.489547] Epoch: 6301  Learning Done!!
[2018-01-20 02:06:59.489547] Epoch: 6301 cost = 0.688612461 acc = 0.508257687
[2018-01-20 02:11:03.636903] Epoch: 6401  Learning Done!!
[2018-01-20 02:11:03.636903] Epoch: 6401 cost = 0.688706517 acc = 0.506996989
[2018-01-20 02:15:07.507433] Epoch: 6501  Learning Done!!
[2018-01-20 02:15:07.507433] Epoch: 6501 cost = 0.688677847 acc = 0.507816434
[2018-01-20 02:19:11.644103] Epoch: 6601  Learning Done!!
[2018-01-20 02:19:11.644103] Epoch: 6601 cost = 0.688548744 acc = 0.508131623
[2018-01-20 02:23:15.527380] Epoch: 6701  Learni

[2018-01-20 05:58:54.714945] Epoch: 12001  Learning Done!!
[2018-01-20 05:58:54.714945] Epoch: 12001 cost = 0.688622713 acc = 0.507438242
[2018-01-20 06:02:59.329853] Epoch: 12101  Learning Done!!
[2018-01-20 06:02:59.329853] Epoch: 12101 cost = 0.688820958 acc = 0.507312179
[2018-01-20 06:07:03.201388] Epoch: 12201  Learning Done!!
[2018-01-20 06:07:03.216988] Epoch: 12201 cost = 0.689113259 acc = 0.507312179
[2018-01-20 06:11:07.125175] Epoch: 12301  Learning Done!!
[2018-01-20 06:11:07.125175] Epoch: 12301 cost = 0.688798189 acc = 0.507186055
[2018-01-20 06:15:11.359287] Epoch: 12401  Learning Done!!
[2018-01-20 06:15:11.359287] Epoch: 12401 cost = 0.688586891 acc = 0.507312179
[2018-01-20 06:19:15.383386] Epoch: 12501  Learning Done!!
[2018-01-20 06:19:15.383386] Epoch: 12501 cost = 0.689042211 acc = 0.507059991
[2018-01-20 06:23:19.580059] Epoch: 12601  Learning Done!!
[2018-01-20 06:23:19.580059] Epoch: 12601 cost = 0.688793182 acc = 0.507186055
[2018-01-20 06:27:23.537741] Epoch

[2018-01-20 10:03:02.388066] Epoch: 18001  Learning Done!!
[2018-01-20 10:03:02.388066] Epoch: 18001 cost = 0.689164639 acc = 0.506744802
[2018-01-20 10:07:06.505748] Epoch: 18101  Learning Done!!
[2018-01-20 10:07:06.505748] Epoch: 18101 cost = 0.688860118 acc = 0.506807864
[2018-01-20 10:11:10.378442] Epoch: 18201  Learning Done!!
[2018-01-20 10:11:10.378442] Epoch: 18201 cost = 0.688914597 acc = 0.507186055
[2018-01-20 10:15:14.701119] Epoch: 18301  Learning Done!!
[2018-01-20 10:15:14.701119] Epoch: 18301 cost = 0.689182639 acc = 0.506933928
[2018-01-20 10:19:18.894852] Epoch: 18401  Learning Done!!
[2018-01-20 10:19:18.894852] Epoch: 18401 cost = 0.689304948 acc = 0.506933928
[2018-01-20 10:23:22.783931] Epoch: 18501  Learning Done!!
[2018-01-20 10:23:22.783931] Epoch: 18501 cost = 0.689213276 acc = 0.506933928
[2018-01-20 10:27:27.047255] Epoch: 18601  Learning Done!!
[2018-01-20 10:27:27.047255] Epoch: 18601 cost = 0.688832581 acc = 0.507059991
[2018-01-20 10:31:30.924363] Epoch

[2018-01-20 14:07:13.308442] Epoch: 24001  Learning Done!!
[2018-01-20 14:07:13.308442] Epoch: 24001 cost = 0.689530194 acc = 0.506744802
[2018-01-20 14:11:18.174370] Epoch: 24101  Learning Done!!
[2018-01-20 14:11:18.174370] Epoch: 24101 cost = 0.689401805 acc = 0.506933928
[2018-01-20 14:15:22.931454] Epoch: 24201  Learning Done!!
[2018-01-20 14:15:22.931454] Epoch: 24201 cost = 0.689226866 acc = 0.506933928
[2018-01-20 14:19:27.777161] Epoch: 24301  Learning Done!!
[2018-01-20 14:19:27.777161] Epoch: 24301 cost = 0.689413249 acc = 0.507186055
[2018-01-20 14:23:32.977038] Epoch: 24401  Learning Done!!
[2018-01-20 14:23:32.977038] Epoch: 24401 cost = 0.690384150 acc = 0.506303608
[2018-01-20 14:27:38.371397] Epoch: 24501  Learning Done!!
[2018-01-20 14:27:38.371397] Epoch: 24501 cost = 0.689499438 acc = 0.507059991
[2018-01-20 14:31:44.202896] Epoch: 24601  Learning Done!!
[2018-01-20 14:31:44.202896] Epoch: 24601 cost = 0.689606786 acc = 0.507564306
[2018-01-20 14:35:49.494430] Epoch

...

[2018-01-21 02:21:58.912220] Epoch: 42001  Learning Done!!
[2018-01-21 02:21:58.912220] Epoch: 42001 cost = 0.688625813 acc = 0.508951068
[2018-01-21 02:26:03.286076] Epoch: 42101  Learning Done!!
[2018-01-21 02:26:03.286076] Epoch: 42101 cost = 0.689006269 acc = 0.508572876
[2018-01-21 02:30:07.433579] Epoch: 42201  Learning Done!!
[2018-01-21 02:30:07.433579] Epoch: 42201 cost = 0.688828945 acc = 0.509329319
[2018-01-21 02:34:11.275689] Epoch: 42301  Learning Done!!
[2018-01-21 02:34:11.275689] Epoch: 42301 cost = 0.689134121 acc = 0.507942498
[2018-01-21 02:38:15.169979] Epoch: 42401  Learning Done!!
[2018-01-21 02:38:15.169979] Epoch: 42401 cost = 0.689481497 acc = 0.507753432
[2018-01-21 02:42:19.295480] Epoch: 42501  Learning Done!!
[2018-01-21 02:42:19.295480] Epoch: 42501 cost = 0.689148903 acc = 0.508257687
[2018-01-21 02:46:23.174933] Epoch: 42601  Learning Done!!
[2018-01-21 02:46:23.174933] Epoch: 42601 cost = 0.689146578 acc = 0.507753432
[2018-01-21 02:50:27.302586] Epoch

[2018-01-21 06:26:08.140400] Epoch: 48001  Learning Done!!
[2018-01-21 06:26:08.140400] Epoch: 48001 cost = 0.689973950 acc = 0.507753432
[2018-01-21 06:30:12.757899] Epoch: 48101  Learning Done!!
[2018-01-21 06:30:12.757899] Epoch: 48101 cost = 0.689849317 acc = 0.507816434
[2018-01-21 06:34:16.935390] Epoch: 48201  Learning Done!!
[2018-01-21 06:34:16.935390] Epoch: 48201 cost = 0.689743578 acc = 0.508194625
[2018-01-21 06:38:20.764125] Epoch: 48301  Learning Done!!
[2018-01-21 06:38:20.764125] Epoch: 48301 cost = 0.689357102 acc = 0.507564306
[2018-01-21 06:42:24.940030] Epoch: 48401  Learning Done!!
[2018-01-21 06:42:24.940030] Epoch: 48401 cost = 0.689635336 acc = 0.508194625
[2018-01-21 06:46:29.366954] Epoch: 48501  Learning Done!!
[2018-01-21 06:46:29.366954] Epoch: 48501 cost = 0.689754725 acc = 0.507690370
[2018-01-21 06:50:33.590845] Epoch: 48601  Learning Done!!
[2018-01-21 06:50:33.590845] Epoch: 48601 cost = 0.689428747 acc = 0.509770572
[2018-01-21 06:54:38.014695] Epoch

[2018-01-21 10:30:37.320871] Epoch: 54001  Learning Done!!
[2018-01-21 10:30:37.320871] Epoch: 54001 cost = 0.689784765 acc = 0.507753432
[2018-01-21 10:34:49.276142] Epoch: 54101  Learning Done!!
[2018-01-21 10:34:49.276142] Epoch: 54101 cost = 0.689634204 acc = 0.507753432
[2018-01-21 10:39:02.536470] Epoch: 54201  Learning Done!!
[2018-01-21 10:39:02.536470] Epoch: 54201 cost = 0.689553022 acc = 0.507879496
[2018-01-21 10:43:16.130223] Epoch: 54301  Learning Done!!
[2018-01-21 10:43:16.130223] Epoch: 54301 cost = 0.689406216 acc = 0.507879496
[2018-01-21 10:47:31.068720] Epoch: 54401  Learning Done!!
[2018-01-21 10:47:31.068720] Epoch: 54401 cost = 0.689836621 acc = 0.507438242
[2018-01-21 10:51:46.923248] Epoch: 54501  Learning Done!!
[2018-01-21 10:51:46.923248] Epoch: 54501 cost = 0.689456940 acc = 0.507942498
[2018-01-21 10:56:03.311376] Epoch: 54601  Learning Done!!
[2018-01-21 10:56:03.311376] Epoch: 54601 cost = 0.689561009 acc = 0.508005559
[2018-01-21 11:00:21.409678] Epoch

[2018-01-21 14:46:54.777573] Epoch: 60001  Learning Done!!
[2018-01-21 14:46:54.777573] Epoch: 60001 cost = 0.689806104 acc = 0.507249117
[2018-01-21 14:51:12.953449] Epoch: 60101  Learning Done!!
[2018-01-21 14:51:12.953449] Epoch: 60101 cost = 0.689525008 acc = 0.506996989
[2018-01-21 14:55:30.517777] Epoch: 60201  Learning Done!!
[2018-01-21 14:55:30.517777] Epoch: 60201 cost = 0.689437926 acc = 0.507059991
[2018-01-21 14:59:48.436772] Epoch: 60301  Learning Done!!
[2018-01-21 14:59:48.436772] Epoch: 60301 cost = 0.689882457 acc = 0.506807864
[2018-01-21 15:04:06.817047] Epoch: 60401  Learning Done!!
[2018-01-21 15:04:06.817047] Epoch: 60401 cost = 0.689365625 acc = 0.506303608
[2018-01-21 15:08:24.408734] Epoch: 60501  Learning Done!!
[2018-01-21 15:08:24.408734] Epoch: 60501 cost = 0.689974070 acc = 0.506429672
[2018-01-21 15:12:41.980408] Epoch: 60601  Learning Done!!
[2018-01-21 15:12:41.980408] Epoch: 60601 cost = 0.689624488 acc = 0.506681800
[2018-01-21 15:16:59.203746] Epoch

[2018-01-21 19:03:02.858360] Epoch: 66001  Learning Done!!
[2018-01-21 19:03:02.873960] Epoch: 66001 cost = 0.689696133 acc = 0.506807864
[2018-01-21 19:07:19.547061] Epoch: 66101  Learning Done!!
[2018-01-21 19:07:19.547061] Epoch: 66101 cost = 0.689852715 acc = 0.506555736
[2018-01-21 19:11:35.513586] Epoch: 66201  Learning Done!!
[2018-01-21 19:11:35.513586] Epoch: 66201 cost = 0.689627171 acc = 0.506618738
[2018-01-21 19:15:51.860285] Epoch: 66301  Learning Done!!
[2018-01-21 19:15:51.860285] Epoch: 66301 cost = 0.689723253 acc = 0.506618738
[2018-01-21 19:20:08.307049] Epoch: 66401  Learning Done!!
[2018-01-21 19:20:08.307049] Epoch: 66401 cost = 0.689555764 acc = 0.506618738
[2018-01-21 19:24:24.847564] Epoch: 66501  Learning Done!!
[2018-01-21 19:24:24.847564] Epoch: 66501 cost = 0.689594090 acc = 0.506618738
[2018-01-21 19:28:41.378062] Epoch: 66601  Learning Done!!
[2018-01-21 19:28:41.378062] Epoch: 66601 cost = 0.689187229 acc = 0.506807864
[2018-01-21 19:32:57.382586] Epoch

In [17]:
     loss, acc = sess.run([cost, accuracy], feed_dict= {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
     print('training_epochs:', '%03d'%(training_epochs), 'cost=', '{:.9f}'.format(loss),
                 'acc=', '{:.9f}'.format(acc))

training_epochs: 70000 cost= 0.689259589 acc= 0.508446813


In [18]:
# Let's see if we can predict
prediction = tf.argmax(hypothesis, 1)
correction_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))
pred = sess.run(prediction, feed_dict={X1: x1_data,X2:x2_data, keep_prob: 1})
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y

[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True

[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 Tru

[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0

[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 T

[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True 

[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0

[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 

[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[False] Prediction: 0 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 

In [ ]:
# Let's see if we can predict
#prediction = tf.cast(hypothesis > 0.5, dtype=tf.int32)
prediction = hypothesis
pred = sess.run(prediction, feed_dict={X1: x1_data,X2:x2_data, keep_prob: 1})


In [24]:
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 1
[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 1
[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 0
[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 0
[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 0
[[False False]] Prediction: [ 0.60425806  0.39574191] True Y: 0
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 1
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 0
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 0
[[False False]] Prediction: [ 0.60489303  0.39510694] True Y: 0
[[False False]] Prediction: [ 0.60489303

In [ ]:
print(pred)